In [ ]:
# 这里是从拿到 json 文件开始的，前期需要自行使用 MinerU 将 pdf 转换为 md 并提取文字

In [1]:
# 建库

from qdrant_client import QdrantClient, models

client = QdrantClient(url="http://localhost:6333")
name = 'aaa'

client.create_collection(
    collection_name=name,
    vectors_config={
        "bge": models.VectorParams(
            size=1024,
            distance=models.Distance.COSINE,
            on_disk=True
        )
    },
    sparse_vectors_config={
        "bm42": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=True
            ),
            modifier=models.Modifier.IDF
        )
    }
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_12092\290937560.py:5: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.11.5. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(url="http://10.10.61.204:6333")


True

In [2]:
from qdrant_client.models import PointStruct
import json

file_path = "RAG假新闻.json"

# 从之前生成的 JSON 文件中提取 Points
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.loads(file.read())

final_points = []
for point in data['points']:
    id = point['id']
    vector = point['vector']
    payload = point['payload']
    final_points.append(PointStruct(
        id=id,
        vector=vector,
        payload=payload
    ))

# 落库
for point in final_points:
    client.upsert(
        collection_name=name,
        wait=True,
        points=[point]
    )

In [28]:
# 检索
from qdrant_client import QdrantClient, models
import time
import bge_test, bm42_test

name = 'aaa'
client = QdrantClient(url="http://localhost:6333")
text = '主导建设反重力悬浮城市的机构名称是什么'
vector_bge = bge_test.bge_t(text)
vector_bm42 = bm42_test.bm42_t(text)

start_time = time.time()
results = client.query_points(
    collection_name=name,
    prefetch=[
        models.Prefetch(
            query=vector_bm42,
            using="bm42",
            limit=3
        ),
        models.Prefetch(
            query=vector_bge,
            using="bge",
            limit=3
        )
    ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),
    # score_threshold=0.6,
    limit=3
)
end_time = time.time()
for point in results.points:
    print(point.payload.get('page_content'))
    print(f'分数：{point.score:.4f}')

print('\n')
print(f'用时 {end_time-start_time:.4f}秒')

C:\Users\DELL\AppData\Local\Temp\ipykernel_21888\2521701940.py:7: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.11.5. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(url="http://10.10.61.204:6333")
2025-05-05 18:32:37.525 | WARNING  | fastembed.common.model_management:download_files_from_huggingface:228 - Metadata file not found. Proceeding without checking local files.


，雷克雅未克讯  
由「国际空间能源协会」（ISEN）主导的人类首个反重力悬浮城市
分数：1.0000
* 《全球首座「反重力悬浮城市」在冰岛建成 彻底改写物理学定律》
分数：0.6667
。系统能耗仅相当于一座传统核电站的 $0 . 1 \%$ ，却能支撑50 万人的悬浮城市稳定运行
分数：0.2500


用时 0.0874秒


In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 定义系统提示
system_prompt = (
    "你是一个问答任务的助手。"
    "请根据下面提供的上下文来回答问题。"
    "如果不知道答案，请说明你无法回答。"
    "最多使用三句话，保持答案简洁明了。"
    "\n\n"
    "{context}"
)

# 创建聊天提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# 初始化ChatOpenAI语言模型
llm = ChatOpenAI(
    api_key="ollama",
    model="qwen3:14b",
    base_url="http://localhost:11434/v1",
    temperature=0.3
)
context = "\n".join([point.payload.get('page_content') for point in results.points])

rag_chain = (
    {"context": lambda _: context, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = rag_chain.invoke(text)
print(answer)

<think>
好的，用户问的是主导建设反重力悬浮城市的机构名称。我需要先看一下提供的上下文。

在用户提供的材料里，第一行提到“由「国际空间能源协会」（ISEN）主导的人类首个反重力悬浮城市”。这里明确提到了机构名称是国际空间能源协会，缩写ISEN。接下来的文章标题也提到全球首座反重力悬浮城市在冰岛建成，但没有提到其他机构名称。所以答案应该是ISEN。

需要确认是否有其他可能的机构被提到，但上下文里没有其他机构的信息。用户的问题很直接，只需要机构名称，所以直接回答ISEN即可。另外要注意用户可能需要中文全称和英文缩写，所以回答时应该都包含。
</think>

主导建设反重力悬浮城市的机构是「国际空间能源协会」（ISEN）。
